In [134]:
import pandas as pd
import numpy as np
import sklearn as sl
import re


from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [135]:
data = pd.read_csv('/home/maxim/DS/Phase_0/Phase_0_final_proj/Phase0-Project/tmp_data/Data_for_coding')
data.drop(columns=['Unnamed: 0', 'Медианная_цена', 'Залог'], inplace = True)

In [136]:
set(data['Парковка'])

{'многоуровневая', 'на крыше', 'наземная', 'открытая', 'подземная'}

# Target Encoder

In [137]:

class TargetEncoderRooms:
    def __init__(self, smoothing=1.0):
        self.smoothing = smoothing
        self.mapping = {}
        self.global_mean = None  # Добавляем атрибут для глобального среднего
        
    def fit_transform(self, X, y):
        self.global_mean = y.mean()  # Сохраняем глобальное среднее
        self.mapping = self._calculate_means(X, y)
        return self._apply_encoding(X)
        
    def transform(self, X):
        return self._apply_encoding(X)
        
    def _calculate_means(self, X, y):
        global_mean = y.mean()
        means = {}
        
        for room in X.unique():
            room_mean = y[X == room].mean()
            n = (X == room).sum()
            
            # Сглаживание для предотвращения переобучения
            means[room] = (room_mean * n + global_mean * self.smoothing) / (n + self.smoothing)
            
        return means
        
    def _apply_encoding(self, X):
        # Используем сохраненное глобальное среднее
        return X.map(self.mapping).fillna(self.global_mean)


# Создаем экземпляр кодировщика
encoder = TargetEncoderRooms(smoothing=1.0)




# LabelEncoder()

In [138]:
le = LabelEncoder()

# OneHotEncoder

In [139]:
one = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' для dummy encoding


In [140]:
data['Количество комнат'] = encoder.fit_transform(data['Количество комнат'], data['Цена'])

In [141]:
le.fit(data.Балкон)
data['Балкон'] = le.transform(data.Балкон)


In [142]:
one = OneHotEncoder(sparse_output=False)

# Применяем кодирование
encoded_data = one.fit_transform(data[['Окна']])
print("Закодированные данные:\n", encoded_data)

# Получаем названия новых столбцов
feature_names = one.get_feature_names_out(['Окна'])
print("\nНазвания новых столбцов:", feature_names)

# Создаем новый DataFrame
temp = pd.DataFrame(encoded_data, columns=feature_names)

# Объединяем и удаляем исходный столбец
data = pd.concat([data, temp], axis=1)
data.drop(columns=['Окна'], inplace=True)

Закодированные данные:
 [[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]

Названия новых столбцов: ['Окна_Во двор' 'Окна_На улицу' 'Окна_На улицу и двор']


In [144]:
one = OneHotEncoder(sparse_output=False)

# Применяем кодирование
encoded_data = one.fit_transform(data[['Парковка']])
print("Закодированные данные:\n", encoded_data)

# Получаем названия новых столбцов
feature_names = one.get_feature_names_out(['Парковка'])
print("\nНазвания новых столбцов:", feature_names)

# Создаем новый DataFrame
temp = pd.DataFrame(encoded_data, columns=feature_names)

# Объединяем и удаляем исходный столбец
data = pd.concat([data, temp], axis=1)
data.drop(columns=['Парковка'], inplace=True)

Закодированные данные:
 [[0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]

Названия новых столбцов: ['Парковка_многоуровневая' 'Парковка_на крыше' 'Парковка_наземная'
 'Парковка_открытая' 'Парковка_подземная']


In [ ]:
# Применяем кодирование
encoded_data = one.fit_transform(data[['Можно с детьми/животными']])

# Получаем названия новых столбцов
feature_names = one.get_feature_names_out(['Можно с детьми/животными'])

# Создаем новый DataFrame
temp = pd.DataFrame(encoded_data, columns=feature_names)
data = pd.concat([data, temp], axis=1)

data.drop(columns=['Можно с детьми/животными'], inplace=True)

In [ ]:
data['Тип'] = le.fit_transform(data['Тип'])


In [145]:
data

,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Цена,Ремонт,"Площадь комнат, м2",...,"Высота потолков, м",Лифт,Окна_Во двор,Окна_На улицу,Окна_На улицу и двор,Парковка_многоуровневая,Парковка_на крыше,Парковка_наземная,Парковка_открытая,Парковка_подземная
0,273973191,73234.305229,Квартира,м. Смоленская (9 мин пешком),"Москва, Новинский бульвар, 18С1",120.0,"5/10, Сталинский",130000.0,NaN,45 25 25,...,3.00,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,273999490,73234.305229,Квартира,м. Смоленская (2 мин пешком),"Москва, Новинский бульвар, 7",80.0,"4/8, Кирпичный",100000.0,NaN,NaN,...,NaN,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,274995950,53953.881519,Квартира,м. Арбатская (10 мин пешком),"Москва, улица Новый Арбат, 15",30.0,19/26,120000.0,NaN,NaN,...,NaN,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,271265359,53953.881519,Квартира,м. Смоленская (5 мин пешком),"Москва, улица Арбат, 45/24",60.0,5/6,90000.0,NaN,18-25,...,3.20,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,274223065,53953.881519,Квартира,м. Смоленская (8 мин пешком),"Москва, Большой Николопесковский переулок, 3",60.0,3/6,120000.0,NaN,NaN,...,NaN,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17095,202336163,38380.317245,Квартира,м. Солнцево (10 мин пешком),"Москва, Солнцевский проспект, 32, ш. Боровское...",32.0,"9/9, Панельный",35000.0,NaN,NaN,...,2.64,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
17096,274654844,38380.317245,Квартира,м. Солнцево (7 мин пешком),"Москва, Производственная улица, 8к1, ш. Боровс...",38.7,"5/18, Монолитный",45000.0,NaN,NaN,...,NaN,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
17097,268679909,53953.881519,Квартира,м. Солнцево (6 мин пешком),"Москва, Боровский проезд, 11",43.1,"5/5, Кирпичный",50000.0,NaN,NaN,...,NaN,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
17098,274807525,53953.881519,Квартира,м. Солнцево (11 мин пешком),"Москва, улица Богданова, 6к1, ш. Боровское (3 ...",52.5,"8/23, Монолитный",55000.0,NaN,NaN,...,2.65,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
data = data.rename(columns={
    'ID объявления': 'ID_coded',
    'Количество комнат': 'Rooms_coded',
    "Тип": "Type_coded"
})